# Redes neuronales artificiales y tasa de cambio: un enfoque desde la microestructura financiera

## 1) Metodología

#### Flujo de órdenes

O'Harad (1995) define la microestructura de un mercado como los mecanismos específicos a ese mercado que inciden en el proceso de formación de precios. Evans & Lyons (2001) demuestran la importancia de la variable de flujo de órdenes en la formación de precios de las divisas en un mercado cambiario. El flujo de órdenes es una variable de la microestructura de este mercado en específico ya que solo está disponible para los agentes de este mercado de manera inmediata (se muestra usualmente en las pantallas de los agentes).

El flujo de órdenes se define como el flujo acumulado de las transacciones *marcadas* en un mercado cambiario. Es decir, cuando una transacción se inicia en el mercado, un trader anuncia a través de un 'quote' su intención de vender o comprar divisa extranjera. Luego, otro agente actúa sobre este anuncio y la transacción se concreta. Definimos el primer trader como el 'market maker' y el segundo como el 'agresor' de este último. De esta manera, si el 'agresor' compra divisa extranjera (el 'market maker' vende), la transacción es *marcada* con un $+1$. De lo contrario, si el 'agresor' vende divisa extranjera, actuando sobre el anuncio de un 'market maker', la transacción sería *marcada* con un $-1$.

Así, si por ejemplo un agente actúa sobre 10 anuncios en una hora, 4 en los que compra y 6 en los que vende, se dice que tiene un flujo de órdenes neto de $ + 4 - 6 = -2$ para esa hora. 

Es importante recalcar que el flujo de órdenes **no** es una representación de la oferta y la demanda en el mercado ya que solamente tiene en cuenta las transacciones que efectivamente se realizan y no los prospectos de transacciones posibles.

El flujo de órdenes en una hora determinada $k$ es entonces la sumatoria del flujo de órdenes neto de todos los agentes en el mercado, 
$$ X_{k} = \sum_{i=1}^N T_{i,k} $$ donde $T_{i,k}$ representa el flujo de órdenes neto del trader $i$ en la hora $k$.

#### Redes neuronales artificiales recurrentes

Con el aumento en la disponibilidad de los datos y la capacidad de cómputo, los modelos de tipo redes neuronales artificiales han empezado a ser cada vez más pertinentes a la hora de modelar relaciones entre variables matemáticas altamente no lineales. 
Más recientemente, este tipo de modelos han empezado a ser usados en contextos de predicción de precios en mercados financieros (Financial Markets Department Bank of Canada, 2000).

Este tipo de modelos utilizan una función de costo para determinar los pesos asignados a las diferentes variables independientes, las cuales son llamadas 'capas de entrada'. Este proceso se ve reflejado en las 'capas ocultas' de la red neuronal. De esta manera, cada capa de este tipo representa una forma funcional distinta e independiente de las otras que sirve para la predicción final de la 'capa de salida'. Este proceso se puede ver reflejado en la imagen adjunta.


![Basic Neural Net](neuralnet.png)


Por otro lado, a diferencia de una red neuronal sencilla, las redes neuronales artificiales recurrentes poseen una capa adicional llamada 'celda estado' que prioriza información pasada relevante para la predicción en el siguiente periodo. Es decir, este tipo de redes neuronales utilizan información pasada (por ejemplo, información en $t-10$) como 'contexto' pertinente y relevante para la predicción de $t+1$.
Sin embargo, este modelo tiene el problema de que puede llegar a poner demasiado énfasis en la información pasada, creado así dependencias de largo plazo. Para solucionar este problema, Hochreiter & Schmidhuber (1997) introdujeron las redes neuronales de tipo Long Short-Term Memory (LSTM) que le permitía al modelo 'olvidar' la información pasada que no era relevante, en los casos donde es más relevante aquella que ocurrió hace no mucho tiempo.

![Memory Cell](memory_cell.png)

En conclusión, para modelar los retornos en el siguiente periodo $r_{t+1}$, se implementará un modelo de tipo LSTM que tendrá como variables independientes principales los retornos en los dos últimos periodos $r_{t}$ y $r_{t-1}$, el precio de cierre $P_{t}$ y $P_{t-1}$ y el flujo de órdenes $ X_{t}$ y $ X_{t-1}$.
Además se pretende controlar por la volatilidad $\sigma_{t}$ y $\sigma_{t-1}$, por los anuncios macroeconómicos observados en el mercado $A_{t}$ y $A_{t-1}$ y el número total de transacciones como proxy del volúmen total transado $N_{t}$ y $N_{t-1}$.

De esta manera, asumimos que los retornos son función de las variables determinadas anteriormente más una variable aleatoria de error $v \sim N(0,1)$:

$$r_{t+1} 
= m(r_{t}, r_{t-1}, X_{t}, X_{t-1}, N_{t}, 
N_{t-1}, \sigma_{t}, \sigma_{t-1}, A_{t}, A_{t-1}, v_{t})$$




El algoritmo hará entonces la estimación formal de la siguiente función a través de un modelo de redes neuronales artificiales recurrentes de tipo LSTM:
$$r_{t+1} = f[\sum_{k=1}^K \alpha_{k} g(\sum_{j=1}^J \beta_{k, j} X_{j} )] $$ donde $r_{t+1}$ son los retornos en el siguiente periodo, K es el número de neuronas en la  `hidden layer', J es el número de variables independientes, $\alpha_{k}$ y $\beta_{k, j}$ son los pesos para las variables y $X_{j}$ son los valores de la j-ésima variable independiente.

## 2) Datos

In [14]:
import pandas as pd 
data = pd.read_csv("datos_py_3.csv", parse_dates=["date"])

In [7]:
data.head()

,date,price,last_sale_price_in _hour,average_price,sd_of_purchase_prices_in_hour,sd_of _sale_prices_in_hour,number_of_trades,number_of_announcements,order_flow,change_price
0,1/05/96 6:00,1.5336,1.5337,1.53365,0.111270,0.191485,11,0,3,0.000000
1,1/05/96 7:00,1.5322,1.5325,1.53235,0.805260,1.019570,48,2,22,-0.000913
2,1/05/96 8:00,1.5300,1.5318,1.53090,0.758654,0.700872,45,0,-3,-0.001437
3,1/05/96 9:00,1.5316,1.5343,1.53295,0.589491,0.633114,53,0,-13,0.001045
4,1/05/96 10:00,1.5350,1.5347,1.53485,0.863529,0.698212,157,0,17,0.002217


In [6]:
data.shape

(1032, 10)

In [11]:
x = pd.to_datetime(x, dayfirst=True)
type(x)

pandas.core.series.Series

In [6]:
from bokeh.plotting import figure, output_file, show


In [11]:
data.describe()

,price,last_sale_price_in _hour,average_price,sd_of_purchase_prices_in_hour,sd_of _sale_prices_in_hour,number_of_trades,number_of_announcements,order_flow,change_price
count,1032.000000,1032.000000,1032.000000,1032.000000,1032.000000,1032.000000,1032.000000,1032.000000,1032.000000
mean,1.509941,1.509881,1.509911,0.650256,0.655849,217.572674,0.890504,-0.303295,-0.000025
std,0.023997,0.024013,0.024003,0.451857,0.448047,147.590221,1.094305,30.902244,0.001258
min,1.467000,1.466900,1.466950,0.000000,0.000000,2.000000,0.000000,-100.000000,-0.012364
25%,1.485000,1.484800,1.484863,0.405025,0.410340,112.000000,0.000000,-18.000000,-0.000592
50%,1.520750,1.520550,1.520675,0.586431,0.587206,200.000000,1.000000,0.000000,0.000000
75%,1.530400,1.530325,1.530337,0.784171,0.798898,288.000000,1.000000,16.000000,0.000524
max,1.548600,1.549500,1.548550,6.964123,6.848557,1091.000000,6.000000,209.000000,0.009472


In [1]:
#Plot

output_file("price_plot.html")
f = figure(logo=None, sizing_mode="scale_width", x_axis_type="datetime")
f.line(data['date'],data['price'])
f.title.text = "Flujo de órdenes vs precio"
show(f)

NameError: name 'output_file' is not defined

## Bibliografía

- O'Hara, Maureen, *Market Microstructure Theory*, Blackwell, Oxford, 1995
- Martin D. D. Evans & Richard K. Lyons, 2001. *Portfolio Balance, Price Impact, and Secret Intervention*, NBER Working Papers 8356, National Bureau of Economic Research
- Financial Markets Department Bank of Canada (2000). *The Application of Artificial Neural Networks to Exchange Rate Forecasting: The Role of Market Microstructure Variables*. Ottawa, Ontario, Canada.
- Hochreiter & Schmidhuber (1997) *Long Short-Term Memory*, Neural Computation  